## Import Packages

In [1]:
from mstrio.connection import Connection
from mstrio.admin.server import Cluster, ServerSettings
from mstrio.admin.application import Environment, Application, compare_application_settings
from mstrio.admin.usergroup import UserGroup, list_usergroups
from mstrio.admin.user import User, list_users
from mstrio.admin.schedule import ScheduleManager
from mstrio.admin.subscription.subscription import EmailSubscription, Subscription
from mstrio.admin.subscription.subscription_manager import SubscriptionManager
from mstrio.admin.subscription.content import Content
from mstrio.admin.privilege import Privilege

from mstrio.library import Library
from mstrio.admin.dossier import list_dossiers
import pandas as pd

# PROD ENVIRONMENT
base_url_prod = "https://env-xxxxxx.customer.cloud.microstrategy.com/MicroStrategyLibrary/api/"
username_prod = "username"
password_prod = "passwd"

# DEV ENVIRONMENT
base_url_dev = "https://env-xxxxxx.customer.cloud.microstrategy.com/MicroStrategyLibrary/api/"
username_dev = "username"
password_dev = ""

mstr_tutorial_id = 'B7CA92F04B9FAE8D941C3E9B7E0CD754'

In [3]:
#df = pd.DataFrame(quotes_final)
#df.to_csv("schedule.csv", index=False, encoding='utf-8', sep="#", quoting=csv.QUOTE_NONE, escapechar="\\", header=False)

import csv
# Define Data
RESULTS = ['apple','cherry','orange','pineapple','strawberry']

# Open File
resultFyle = open("output.csv",'w')

# Write data to file
for r in RESULTS:
    resultFyle.write(r + "\n")
resultFyle.close()

## Connect to Environment

In [2]:
conn_prod = Connection(base_url_prod, username_prod, password_prod, login_mode=1, project_name="Tutorial_Prod")
env_prod = Environment(connection=conn_prod)
tutorial_prod = Application(conn_prod, name="Tutorial_Prod")


Connection to MicroStrategy Intelligence Server has been established.
Application object named: 'Tutorial_Prod' with ID: 'B7CA92F04B9FAE8D941C3E9B7E0CD754'


In [3]:
conn_dev = Connection(base_url_dev, username_dev, password_dev, login_mode=1, project_name="Tutorial_Dev")
env_dev = Environment(connection=conn_dev)
tutorial_dev = Application(conn_dev, name="Tutorial_Dev")


Connection to MicroStrategy Intelligence Server has been established.
Application object named: 'Tutorial_Dev' with ID: 'B7CA92F04B9FAE8D941C3E9B7E0CD754'


***

***

# Server and Application

### See which services are running on the environment

In [4]:
cluster = Cluster(connection=conn_dev)
node_name = 'env-239822laiouse1'

In [7]:
cluster.nodes_topology()

,displayName,id,env-239822laiouse1
0,Intelligence,MicroStrategy-Intelligence-Server,Running
1,Export,MicroStrategy-PDFExport-Service,Running
2,Telemetry,Apache-Kafka,Stopped
3,Telemetry Manager,Apache-ZooKeeper,Running
4,Collaboration,MicroStrategy-Collaboration-Server,Running
5,Library,MicroStrategy-Library-REST-Server,Running
6,Web,MicroStrategy-Web-Server-ASP,Not Available
7,Mobile,MicroStrategy-Mobile-Server-ASP,Not Available
8,Identity,MicroStrategy-Usher-Security-Server,Not Available
9,Identity Gateway,MicroStrategy-Usher-Gateway-Server,Not Available


### Stop service

In [6]:
cluster.stop(service='Apache-Kafka', nodes=[node_name], login="mstr", passwd="8OT1CtHe21XJ")

Request to stop Apache-Kafka was sent for node(s) env-239822laiouse1.


### Start service

In [8]:
cluster.start(service='Apache-Kafka', nodes=[node_name], login="mstr", passwd="8OT1CtHe21XJ")

Request to start Apache-Kafka was sent for node(s) env-239822laiouse1.


### See which applications are on the environment

In [9]:
cluster.list_applications()

[Application(connection, name='Consolidated Education Project', id='CE52831411E696C8BD2F0080EFD5AF44'),
 Application(connection, name='Hierarchies Project', id='B3FEE61A11E696C8BD0F0080EFC58F44'),
 Application(connection, name='Human Resources Analysis Module', id='4BAE16A340B995CAD24193AA3AC15D29'),
 Application(connection, name='Platform Analytics', id='93AF3FB811EB3C461B920080EFA5E68F'),
 Application(connection, name='Relationships Project', id='4C09350211E69712BAEE0080EFB56D41'),
 Application(connection, name='Tutorial_Dev', id='B7CA92F04B9FAE8D941C3E9B7E0CD754')]

### Unload application

In [10]:
app_name = 'Consolidated Education Project'
cluster.unload_application(application_name=app_name)

'Consolidated Education Project' unloaded on node 'env-239822laiouse1'.


### See loaded appliactions

In [11]:
env_dev.list_loaded_applications()

[Application(connection, name='Tutorial_Dev', id='B7CA92F04B9FAE8D941C3E9B7E0CD754'),
 Application(connection, name='Hierarchies Project', id='B3FEE61A11E696C8BD0F0080EFC58F44'),
 Application(connection, name='Human Resources Analysis Module', id='4BAE16A340B995CAD24193AA3AC15D29'),
 Application(connection, name='Relationships Project', id='4C09350211E69712BAEE0080EFB56D41'),
 Application(connection, name='Platform Analytics', id='93AF3FB811EB3C461B920080EFA5E68F')]

### Load application

In [12]:
cluster.load_application(application_name=app_name)

'Consolidated Education Project' loaded on node 'env-239822laiouse1'.


### Export Application and Server Setting to JSON

In [13]:
tutorial_dev = Application(conn)
tutorial_dev.settings.to_json("tutorial_dev_settings.json")

Settings exported to 'tutorial_dev_settings.json'


In [ ]:
env_dev = Cluster(conn)
env_dev.settings.to_json

In [14]:
env_dev_settings = ServerSettings(conn_dev)
env_dev_settings.to_json("env_dev_settings.json")

Settings exported to 'env_dev_settings.json'


### Compare project level settings of tutorial project in development and production environments

In [15]:
compare_application_settings(applications=[tutorial_prod, tutorial_dev], show_diff_only=True)

,setting,Tutorial_Prod,Tutorial_Dev
115,maxJobPerProject,1000,500
116,maxJobPerUserAccount,100,75
117,maxJobPerUserConnection,100,75


### Compare server level settings of two environments

In [18]:
# get settings of development environment
settings_dev = ServerSettings(connection=conn_dev)
setts_props_dev = settings_dev.list_properties()

# get settings of production environment
settings_prod = ServerSettings(connection=conn_prod)
setts_props_prod = settings_prod.list_properties()

# compare settings and save differences in a table of dictionaries
diff_table = []
for key_dev, value_dev in setts_props_dev.items():
    if value_dev != setts_props_prod[key_dev]:
        diff_table.append({'key': key_dev,
                           'value_dev': value_dev,
                           'value_prod': setts_props_prod[key_dev]})
pd.DataFrame.from_dict(diff_table)

,key,value_dev,value_prod
0,historyListRunningStatus,True,False


### Update Server Settings of Dev environment to match Prod environment 

In [17]:
update_dict = {}
for diff in diff_table:
    update_dict[diff['key']] = diff['value_prod']
    
if update_dict:
    settings_dev.alter(**update_dict)
    settings_dev.update()

I-Server settings updated.


### Export Prod Tutorial Project Settings into Dev Tutorial Project

In [19]:
tutorial_prod.settings.to_csv("tutorial_prod_settings.csv")
tutorial_dev.settings.import_from("tutorial_prod_settings.csv")
tutorial_dev.settings.update()

Settings exported to 'tutorial_prod_settings.csv'
Settings imported from 'tutorial_prod_settings.csv'


In [20]:
compare_application_settings(applications=[tutorial_prod, tutorial_dev], show_diff_only=True)

There is no difference in settings between application 'Tutorial_Prod' and remaining applications: '['Tutorial_Dev']'


,setting,Tutorial_Prod,Tutorial_Dev


***

***

***

***

# User and Privileges

### Prepare data to connect to database

In [21]:
import pyodbc # Python library to connect to database

driver = '{ODBC Driver 17 for SQL Server}'
server = '127.0.0.1'
database = 'TmpDB_MSTR'
uid = 'sa'
pwd = 'reallyStrongPwd123'

# prepare string which is used to connect to database
driver = 'DRIVER=' + driver
server = 'SERVER=' + server
database = 'DATABASE=' + database
uid = 'UID=' + uid
pwd = 'PWD=' + pwd
connection_string = ';'.join([driver, server, database, uid, pwd])

### Get newly hired employees from Database and create new user accounts for them. 

In [22]:
query = "SELECT * FROM NEW_HIRE_EMPLOYEE WHERE EMPLOYEE_HIRE_DATE > '12-01-2020';"

with pyodbc.connect(connection_string) as db_conn:
    new_employees = pd.read_sql(query, db_conn)

new_users = []
for _, row in new_employees.iterrows():
    emp_username = row['EMPLOYEE_USER_NAME']
    emp_fullname = row['EMPLOYEE_LEGAL_NAME']
    emp_title = row['EMPLOYEE_TITLE']
    new_usr = User.create(connection=conn_dev, username=emp_username, full_name=emp_fullname,
                          trust_id=emp_username, require_new_password=False)
    new_users.append(new_usr)

Successfully created user named: 'lchase' with ID: 'F7E9C46A11EB59CB75F00080EF05EF16'
Successfully created user named: 'dayers' with ID: 'F812943011EB59CB75F00080EF252E14'
Successfully created user named: 'nmacfarlane' with ID: 'F8307B1C11EB59CB75F00080EFF5CF16'
Successfully created user named: 'mferry' with ID: 'F8519B3011EB59CB75F00080EF558E15'
Successfully created user named: 'cphillips' with ID: 'F8761BE111EB59CB75F00080EFE5AE14'
Successfully created user named: 'bdaniels' with ID: 'F8934D1411EB59CB75F00080EF85EF17'
Successfully created user named: 'brees' with ID: 'F8B2A8BC11EB59CB75F00080EFB54E15'


### Add newly created users to Administrator group

In [23]:
usergroup = UserGroup(connection=conn_dev, name='Administrator')
usergroup.add_users(users=new_users)

UserGroup object named: 'Administrator' with ID: 'E96685CD4E60068559F7DFAC7C2AA851'
Added ['Linda Chase', 'Dylan Ayers', 'Nicky Macfarlane', 'Mark Ferry', 'Chandler Phillips', 'Bradley Daniels', 'Brandon Rees'] user(s) to group Administrator


### Add email addresses with domain 'microstrategy.com' to newly created users

In [24]:
new_users_with_emails = []
for usr in new_users:
    email_address = usr.username + '@microstrategy.com' 
    usr.add_address(name=usr.username, address=email_address)
    new_users_with_emails.append(usr)

Added address 'lchase@microstrategy.com' for user 'Linda Chase'
Added address 'dayers@microstrategy.com' for user 'Dylan Ayers'
Added address 'nmacfarlane@microstrategy.com' for user 'Nicky Macfarlane'
Added address 'mferry@microstrategy.com' for user 'Mark Ferry'
Added address 'cphillips@microstrategy.com' for user 'Chandler Phillips'
Added address 'bdaniels@microstrategy.com' for user 'Bradley Daniels'
Added address 'brees@microstrategy.com' for user 'Brandon Rees'


### List empty usergroups

In [25]:
all_usrgrps = list_usergroups(connection=conn_dev)
empty_usergroups = [usrgrp for usrgrp in all_usrgrps if not usrgrp.list_members()]

In [26]:
for ug in empty_usergroups:
    print(ug.name)

3rd Party Users
AnalyticsServer
API
Architect
CollaborationServer
Desktop
DistributionServer
LDAP Public / Guest
LDAP Users
MicroStrategy Desktop Users
Mobile
Narrowcast System Administrators
Northeast Employees
Public / Guest
Reporter
Second Factor Exempt
Server Bulk Administrators
Server Configuration Administrators
Server Operations Administrators
Server Operations Monitors
Server Resource Settings Administrators
Server Security Administrators
TransactionServer
User Administrators
User Role Bundles
Users from Poland
Users from United States
Warehouse Users
Web
Web Viewers


### Delete empty usergroups

In [27]:
ug = UserGroup(conn_dev, name="Web Viewers")
ug.delete(force=True)

UserGroup object named: 'Web Viewers' with ID: '9B9F35D811EB59B675F00080EFB54F17'
Successfully deleted User Group Web Viewers


True

### Disable few users and delete email addresses of them

In [28]:
usernames = ['jsmith', 'rwilliams', 'mjohnson']
for username in usernames:
    usr = User(conn_dev, username=username)
    usr.alter(enabled=False)
    # remove all addresses from a given user
    if usr.addresses:
        for addr in usr.addresses:
            usr.remove_address(id=addr['id'])



User object named: 'John Smith' with ID: '39EBD0DA11EB59B675F00080EF354E14'
User 'John Smith' has been modified.
Removed address 'jsmith' with id 3A0F286511EB59B615700080EF25AF22 from user 'John Smith'
User object named: 'Robert Williams' with ID: '3A1B229B11EB59B675F00080EF950D13'
User 'Robert Williams' has been modified.
Removed address 'rwilliams' with id 3A3FB9CB11EB59B615700080EF755025 from user 'Robert Williams'
User object named: 'Micheal Johnson' with ID: '3A4DE40B11EB59B675F00080EFF5CE14'
User 'Micheal Johnson' has been modified.
Removed address 'mjohnson' with id 3A6FEBC311EB59B615700080EF45EF22 from user 'Micheal Johnson'


***

***

***

***

***

***

# Subscriptions and Schedules

### Get list of schedules and its properties

In [29]:
schedule_mngr = ScheduleManager(connection=conn_dev)
schedule_mngr.list_schedules()

[{'name': 'All the Time',
  'id': 'FF7BB3C811D501F0C00051916B98494F',
  'description': 'Starts Friday, January 01, 2010 and ends Sunday, May 30, 2010. The schedule will be triggered every day. 1 hours, 0 minutes',
  'scheduleType': 'time_based',
  'scheduleNextDelivery': '2021-01-18T21:00:00+0000',
  'startDate': '2009-12-31',
  'time': {'recurrencePattern': 'daily',
   'execution': {'executionPattern': 'repeat',
    'startTime': '00:00:00',
    'stopTime': '23:59:00',
    'repeatInterval': 60},
   'daily': {'dailyPattern': 'day', 'repeatInterval': 1}},
  'expired': False},
 {'name': 'At Close of Business (Weekday)',
  'id': 'FF7BB3BC11D501F0C00051916B98494F',
  'description': 'Starts Wednesday, February 14, 2001 and has no end. The schedule will be triggered every weekday. It runs at 6:00:00 PM',
  'scheduleType': 'time_based',
  'scheduleNextDelivery': '2021-01-19T18:00:00+0000',
  'startDate': '2001-02-14',
  'time': {'recurrencePattern': 'weekly',
   'execution': {'executionPattern

### Create an email subscription that displays the list of departed users which still need to be removed from user groups

In [30]:
lchase = User(conn_dev, name="Linda Chase")
new_sub = EmailSubscription.create(
    connection=conn_dev, name="Departed Users", email_subject='Remove departed users from usergroups',
    email_message='Users to remove are:', recipients=[lchase.id],
    schedules_ids='FF7BB3B311D501F0C00051916B98494F', application_name="Tutorial_Dev",
    contents=Content(id='FE72F33B11E7F54F000000802F31418D', type=Content.Type.DOSSIER, personalization=Content.Properties()))

    
    
#     application_name="MicroStrategy Tutorial",
    
    
#     contents=Content(id='ABC123ABC123ABC123ABC123ABC12345', type=Content.Type.REPORT),
#     schedules_ids=['ABC123ABC123ABC123ABC123ABC12345'], recipients=['ABC123ABC123ABC123ABC123ABC12345'])

User object named: 'Linda Chase' with ID: 'F7E9C46A11EB59CB75F00080EF05EF16'
Created subscription 'Departed Users' with ID: 'A09E3CDA11EB59CC7CCF0080EF150F16'.


In [34]:
subs_mngr = SubscriptionManager(conn_dev, application_name='Tutorial_Dev')
subs_mngr.list_subscriptions(name="Departed Users")

[{'id': 'A09E3CDA11EB59CC7CCF0080EF150F16',
  'name': 'Departed Users',
  'editable': False,
  'allowDeliveryChanges': False,
  'allowPersonalizationChanges': False,
  'allowUnsubscribe': True,
  'dateCreated': '2021-01-18T20:34:56+0000',
  'dateModified': '2021-01-18T20:36:18+0000',
  'owner': {'id': 'F7E9C46A11EB59CB75F00080EF05EF16', 'name': 'Linda Chase'},
  'schedules': [{'id': 'FF7BB3B311D501F0C00051916B98494F',
    'name': 'Monday Morning',
    'type': 'time_based',
    'nextDelivery': '2021-01-25T04:00:00+0000',
    'expired': False}],
  'contents': [{'id': 'FE72F33B11E7F54F000000802F31418D',
    'name': 'Advanced and Predictive Analytics',
    'type': 'dossier',
    'personalization': {'compressed': False,
     'formatMode': 'DEFAULT',
     'viewMode': 'BOTH',
     'formatType': 'PDF',
     'exportToPdfSettings': {'pageOption': 'PAGE',
      'pageSize': 'letter',
      'orientation': 'automatic',
      'pageDetailLevel': 'overview',
      'includeToc': False,
      'includeHea

In [32]:
new_sub.alter(owner_id = lchase.id)

Updated subscription 'Departed Users' with ID: A09E3CDA11EB59CC7CCF0080EF150F16.


In [33]:
for usr in new_users:
    new_sub.add_recipient(usr.id)

No recipients were added to the subscription.
Updated subscription 'Departed Users' with ID: A09E3CDA11EB59CC7CCF0080EF150F16.
Updated subscription 'Departed Users' with ID: A09E3CDA11EB59CC7CCF0080EF150F16.
Updated subscription 'Departed Users' with ID: A09E3CDA11EB59CC7CCF0080EF150F16.
Updated subscription 'Departed Users' with ID: A09E3CDA11EB59CC7CCF0080EF150F16.
Updated subscription 'Departed Users' with ID: A09E3CDA11EB59CC7CCF0080EF150F16.
Updated subscription 'Departed Users' with ID: A09E3CDA11EB59CC7CCF0080EF150F16.


***

***

***

***

***

***

# Library and Dossier

### Get all dossiers on Server

In [35]:
dss = list_dossiers(conn_dev)
dss

[Dossier(connection, name='MicroStrategy, Ink.', id='0B38895711E79D7ECFF20080AFB1518B'),
 Dossier(connection, name='Rustic', id='11EA496611E751FB1E9A0080EF25C29E'),
 Dossier(connection, name='Waterfall', id='1532D3684B9D88716CFFAA921CA17458'),
 Dossier(connection, name='Hide Overlapping Markers', id='1687A36449FBD4F6CD55A987AD3B98F2'),
 Dossier(connection, name='Graphs and Visualizations', id='27BB740E11E7A7B40A650080EF856B88'),
 Dossier(connection, name='Investment Firm Dossier', id='2C68268711E7933800000080AFB1D2B9'),
 Dossier(connection, name='Maps with Pie and Clusters', id='316401E111E6BD5BAD440080EFE1DC37'),
 Dossier(connection, name='Multinational Bank Dossier', id='32765C5C11E79D5927C10080EF753106'),
 Dossier(connection, name='Area Chart', id='3511C9E011E5983A85B30080EFE560D9'),
 Dossier(connection, name='The 2014 BNY Mellon Boat Race', id='371E250C11E5BEFB00000080EF85899C'),
 Dossier(connection, name='Combo Chart', id='3850EDC911E5A8F4045C0080EF95C044'),
 Dossier(connection, n

### Get info about a particular dossier

In [36]:
dss[6].info

{'id': '316401E111E6BD5BAD440080EFE1DC37',
 'name': 'Maps with Pie and Clusters',
 'description': 'Use this template to create a Dashboard with HTML5 '
                'technology.',
 'type': 'DOCUMENT_DEFINITION',
 'subtype': 14081,
 'ext_type': 0,
 'date_created': '2016-12-15T14:29:36.000+0000',
 'date_modified': '2017-02-06T17:31:55.000+0000',
 'version': 'BF71346D4DEE224730DE05966E9C3A1F',
 'owner': {'name': 'Administrator', 'id': '54F3D26011D2896560009A8E67019608'},
 'view_media': 1879072805,
 'certified_info': {'certified': False},
 'project_id': 'B7CA92F04B9FAE8D941C3E9B7E0CD754',
 'acg': 255}


### Get all certified dossiers from admin library

In [37]:
certified_dss = []
for ds in dss:
    if ds.certified_info['certified'] is True:
        certified_dss.append(ds)
    
certified_dss

[Dossier(connection, name='Campaign Finance', id='0D0CDC8F11E97E8F00000080AFB3A407'),
 Dossier(connection, name='Avg Delay (min) by Airline', id='69F662EE11E9817800000080AFE34811'),
 Dossier(connection, name='Retail Sales Report', id='CF13BCDA409911243371CA980BAAE749'),
 Dossier(connection, name='Slot Machines Dossier', id='43883BAB11E978D61F220080AFB38619')]

### Publish certified dossiers in New Hires' Library

In [38]:
for certified_ds in certified_dss:
    certified_ds.publish(recipients=new_users)

***

***

***

***

***

# Cleaning

### Change Server and Application Settings 

In [ ]:
env_dev_settings.import_from("env_dev_changed.json")
env_dev_settings.update()

In [ ]:
tutorial_dev.settings.import_from("tutorial_dev_changed.json")
tutorial_dev.settings.update()

### Restore users which were disabled/removed

In [ ]:

usrs = {'jsmith':'John Smith', 'rwilliams':'Robert Williams', 'mjohnson':'Micheal Johnson'}

for username, fullname in usrs.items():
    try:
        usr = User.create(conn_dev, username, fullname, require_new_password=False)
    except:
        usr = User(conn_dev, username=username)
        usr.alter(enable=True)
        
    if len(usr.addresses) == 0:
        email_address = usr.username + '@microstrategy.com'
        usr.add_address(name=usr.username, address=email_address)

### Delete user accounts created during execution of this script

In [ ]:
def delete_users(connection, fullname):
    usrs = list_users(connection=connection, name=fullname)
    for u in usrs:
        u.delete(force=True)

In [ ]:
fullnames = ['Linda Chase', 'Dylan Ayers', 'Nicky Macfarlane', 'Mark Ferry', 'Chandler Phillips', 'Bradley Daniels',
             'Brandon Rees']
for fullname in fullnames:
    delete_users(connection=conn_dev, fullname=fullname)

### Restore deleted usergroups

In [ ]:
ug_tmp_names = ["Web Viewers"]
for name in ug_tmp_names:
    UserGroup.create(conn_dev, name)

### Delete subscription

In [ ]:
new_sub.delete()

### Unpublish dossiers

In [ ]:
for certified_ds in certified_dss:
    certified_ds.unpublish(recipients=new_users)

### Close connections to environments

In [ ]:
conn_prod.close()

In [ ]:
conn_dev.close()